In [ ]:
import pandas as pd
import re
import os

from pymongo import MongoClient
from utils import *

### Limpa Dataset de tipos de infrações

In [ ]:
df_tipos_infracoes_2018 = pd.read_excel("datasets/tipos-infracoes-2018.xlsx")
df_tipos_infracoes_2018 = df_tipos_infracoes_2018.rename(columns = {'Unnamed: 0': 'TIPO DE INFRACAO'})

df_tipos_infracoes_2018.head(n = 20)

### Limpa Dataset de acidentes com vítimas fatais

In [ ]:
df_acidentes_fatais_2008_2017 = pd.read_excel("datasets/acidentes/acidentes-fatais-2008-2017.xlsx")
df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop("Fatal", axis = 1)
df_acidentes_fatais_2008_2017['Ano'] = 0


for ano in range(2008, 2018):
    df_acidentes_fatais_2008_2017.loc[df_acidentes_fatais_2008_2017['Ano ' + str(ano)] == 1.0, 'Ano'] = str(ano)
    df_acidentes_fatais_2008_2017 = df_acidentes_fatais_2008_2017.drop('Ano ' + str(ano), axis = 1)

df_acidentes_fatais_2008_2017.head()

### Limpa Dataset de pedestres mortos em trechos não semaforizados

In [ ]:
df_pedestres_mortos_trechos_ns_2008_2017 = pd.read_excel("datasets/pedestres-mortos-trechos-ns-2008-2017.xlsx")

for ano in range(2008, 2018):
    df_pedestres_mortos_trechos_ns_2008_2017 = df_pedestres_mortos_trechos_ns_2008_2017.drop(ano, axis = 1)

df_pedestres_mortos_trechos_ns_2008_2017.head()

### Limpa Dataset de Cursos oferecidos pelo Detran

In [ ]:
df_cursos_oferecidos_detran_2018 = pd.read_excel("datasets/cursos_oferecidos_detran.xlsx")
df_cursos_oferecidos_detran_2018 = df_cursos_oferecidos_detran_2018.fillna(0)

for mes in df_cursos_oferecidos_detran_2018.columns[3:]:
    df_cursos_oferecidos_detran_2018[mes] = df_cursos_oferecidos_detran_2018[mes].astype(int)

df_cursos_oferecidos_detran_2018.head()

### Limpa Dataset de campanhas oferecidas pelo Detran

In [ ]:
df_campanhas_educativas_2012_2019 = pd.read_excel("datasets/campanhas_educativas_2012_2019.xlsx")

for column in df_campanhas_educativas_2012_2019.columns[6:10]:
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].fillna(0)
    df_campanhas_educativas_2012_2019[column] = df_campanhas_educativas_2012_2019[column].astype(int)

df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'] = df_campanhas_educativas_2012_2019['Região Administrativa do Evento da Campanha'].fillna("--")

df_campanhas_educativas_2012_2019.head()

### Limpa e combina Dataset de acidentes em rodovias

In [ ]:
df_acidentes_rodovias_2017 = pd.read_excel("datasets/acidentes/acidentes-rodovias-2017.xlsx")
df_acidentes_rodovias_2017['uf'] = df_acidentes_rodovias_2017['uf'].fillna("--")
df_acidentes_rodovias_2017 = df_acidentes_rodovias_2017[df_acidentes_rodovias_2017['uf'] == 'DF']
df_acidentes_rodovias_2017 = df_acidentes_rodovias_2017.drop(columns = ['id', 'pesid'], axis = 1)

df_acidentes_rodovias_2017.head()

In [ ]:
df_acidentes_rodovias_2018 = pd.read_excel("datasets/acidentes/acidentes-rodovias-2018.xlsx")
df_acidentes_rodovias_2018['uf'] = df_acidentes_rodovias_2018['uf'].fillna("--")
df_acidentes_rodovias_2018 = df_acidentes_rodovias_2018[df_acidentes_rodovias_2018['uf'] == 'DF']
df_acidentes_rodovias_2018 = df_acidentes_rodovias_2018.drop(columns = ['id', 'pesid'], axis = 1)

df_acidentes_rodovias_2018.head()

In [ ]:
df_acidentes_rodovias_2019 = pd.read_excel("datasets/acidentes/acidentes-rodovias-2019.xlsx")
df_acidentes_rodovias_2019['uf'] = df_acidentes_rodovias_2019['uf'].fillna("--")
df_acidentes_rodovias_2019 = df_acidentes_rodovias_2019[df_acidentes_rodovias_2019['uf'] == 'DF']
df_acidentes_rodovias_2019 = df_acidentes_rodovias_2019.drop(columns = ['id', 'pesid'], axis = 1)

df_acidentes_rodovias_2019.head()

In [ ]:
df_acidentes_rodovias_2017_2019 = pd.concat([df_acidentes_rodovias_2017, df_acidentes_rodovias_2018])
df_acidentes_rodovias_2017_2019 = pd.concat([df_acidentes_rodovias_2017_2019, df_acidentes_rodovias_2019])

df_acidentes_rodovias_2017_2019.info()

### Limpa Dataset de infrações DER

In [ ]:
df_autosinfrac_ago_2018 = pd.read_excel("datasets/infracoes/DER/2018_dez_autosinfrac.xlsx")
pattern_localizacao = re.compile("df|df |df-")

localizacao = []

for identificacao in df_autosinfrac_ago_2018['auinf_local_rodovia']:
    print(identificacao)


### Refresh Database

In [ ]:
# Deletes all records, only if necessary
clear_database()

### Salvar e recuperar DataFrames

In [ ]:
dataframe_to_mongo(df_tipos_infracoes_2018, "tipos_infracoes_2018")
dataframe_to_mongo(df_acidentes_fatais_2008_2017, "acidentes_fatais_2008_2017")
dataframe_to_mongo(df_pedestres_mortos_trechos_ns_2008_2017, "pedestres_mortos_trechos_ns_2008_2017")
dataframe_to_mongo(df_cursos_oferecidos_detran_2018, "cursos_oferecidos_detran_2018")
dataframe_to_mongo(df_campanhas_educativas_2012_2019, "campanhas_educativas_detran_2012_2019")
dataframe_to_mongo(df_acidentes_rodovias_2017_2019, "acidentes_rodovias_2017_2019")

In [ ]:
test = mongo_to_dataframe("acidentes_rodovias_2017_2019")
test.head()